In [ ]:
!pip install jiwer evaluate tensorboard datasets transformers librosa accelerate --quiet

In [2]:
# 2. Environment & Login
from huggingface_hub import login
login(token="INSERT_YOUR_HUGGING_FACE_TOKEN_HERE")

In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("openslr", "SLR66", trust_remote_code = True)

# Set a seed for reproducibility
seed = 42

# Split into 80% training and 20% testing
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=seed)

# Combine the train and validation into one
train_data = train_test_split['train']  # 80% training
test_data = train_test_split['test']    # 20% testing

# Check sizes
print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

README.md:   0%|          | 0.00/42.9k [00:00<?, ?B/s]

openslr.py:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4448 [00:00<?, ? examples/s]

Training set size: 3558
Test set size: 890


In [4]:
telugu_special_unwanted_characters = [
    'ౄ',  # Vocalic RR
    'ౢ',  # Vocalic L
    'ౣ',  # Vocalic LL
    'ౠ',  # Long Vocalic RR
    'ఽ',  # Avagraha
    '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯',  # Telugu digits
    'ఀ',  # Telugu Sign Combining Candrabindu Above
    'ౘ',  # Letter TTHA
    'ౙ',  # Letter DDA
    'ౚ',  # Letter RHA
    '౷',  # Vedic Tone
    '‘', '’', '“', '”', '%', '.', ';', '-', ',', '/', '\\', '_', '&',  # Common punctuation
    'G', 'P', 'S', 'e', 'l', 'n', 'r', 't', '\u200c' #Unwanted in the dataset
]

In [5]:
import re
chars_to_remove_regex = f'[{re.escape("".join(telugu_special_unwanted_characters))}]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"])
    return batch

In [6]:
train_data = train_data.map(remove_special_characters)
test_data = test_data.map(remove_special_characters)

Map:   0%|          | 0/3558 [00:00<?, ? examples/s]

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

In [7]:
repo_name = "whisper-small-telugu-k-fold-4"

In [8]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [9]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [10]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/kaarthu2003/whisper-small-telugu-k-fold-4/commit/5b44cb1c09c4564fc2d1fcb884863a67c9d7a0f3', commit_message='Upload tokenizer', commit_description='', oid='5b44cb1c09c4564fc2d1fcb884863a67c9d7a0f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kaarthu2003/whisper-small-telugu-k-fold-4', endpoint='https://huggingface.co', repo_type='model', repo_id='kaarthu2003/whisper-small-telugu-k-fold-4'), pr_revision=None, pr_num=None)

Ensuring Tokenizer is working correctly

In [11]:
input_str = train_data[0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 వారి శ్రేయోభివృద్ధులే నా ఆనందానికి మూలం
Decoded w/ special:    <|startoftranscript|><|te|><|transcribe|><|notimestamps|>వారి శ్రేయోభివృద్ధులే నా ఆనందానికి మూలం<|endoftext|>
Decoded w/out special: వారి శ్రేయోభివృద్ధులే నా ఆనందానికి మూలం
Are equal:             True


In [12]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

In [13]:
print(train_data[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00158691,
       -0.00112915, -0.00115967]), 'sampling_rate': 48000}, 'sentence': 'వారి శ్రేయోభివృద్ధులే నా ఆనందానికి మూలం'}


In [14]:
from datasets import Audio

train_data = train_data.cast_column("audio", Audio(sampling_rate=16000))
test_data = test_data.cast_column("audio", Audio(sampling_rate=16000))

In [15]:
print(train_data[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav', 'array': array([ 6.55302210e-06, -4.48762003e-05, -8.45453615e-05, ...,
       -2.42830161e-03, -1.52032159e-03, -1.23449310e-03]), 'sampling_rate': 16000}, 'sentence': 'వారి శ్రేయోభివృద్ధులే నా ఆనందానికి మూలం'}


In [16]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [17]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names)

Map:   0%|          | 0/3558 [00:00<?, ? examples/s]

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

In [18]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [19]:
model.generation_config.language = "telugu"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [20]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [22]:
!pip install evaluate --quiet


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [23]:
import evaluate

metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer, "cer": cer}

In [25]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [29]:
from sklearn.model_selection import KFold
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

# ——— prerequisites ———
# train_data: your full HuggingFace Dataset for training (e.g. train_data)
# model: your Seq2Seq model instance
# data_collator: your collator function
# compute_metrics: the function returning {"wer":…, "cer":…}
# processor: your feature_extractor/tokenizer object
# repo_name: base output directory name (string)
# seed: an integer for reproducibility

seed = 42
k_folds = 4
kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)

for fold, (train_idx, val_idx) in enumerate(kf.split(train_data), start=1):
    print(f"\n===== Fold {fold}/{k_folds} =====")
    train_ds = train_data.select(train_idx)
    val_ds   = train_data.select(val_idx)

    training_args = Seq2SeqTrainingArguments(
        output_dir=repo_name,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        learning_rate=1e-5,
        warmup_steps=400,
        num_train_epochs=10,
        gradient_checkpointing=True,
        fp16=True,
        eval_strategy="steps",
        eval_steps=400,
        save_steps=800,
        logging_steps=400,
        save_total_limit=2,
        predict_with_generate=True,
        generation_max_length=250,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
    )

    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        processing_class=processor.feature_extractor,
    )

    trainer.train()
    metrics = trainer.evaluate()
    print(
        f"Fold {fold} — eval_wer: {metrics['eval_wer']:.4f}, "
        f"eval_cer: {metrics['eval_cer']:.4f}"
    )

    # free up GPU memory between folds
    torch.cuda.empty_cache()


===== Fold 1/4 =====


Step,Training Loss,Validation Loss,Wer,Cer
400,0.536000,0.140445,0.452323,0.092056
800,0.055700,0.125187,0.374305,0.074552
1200,0.009800,0.143755,0.350967,0.065166
1600,0.001100,0.151775,0.328295,0.062121


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 2

Fold 1 — eval_wer: 0.3283, eval_cer: 0.0621

===== Fold 2/4 =====


Step,Training Loss,Validation Loss,Wer,Cer
400,0.031900,0.007049,0.032294,0.005796
800,0.010600,0.009895,0.063019,0.010984
1200,0.002600,0.006829,0.043283,0.006661
1600,0.000300,0.005923,0.035210,0.005668


...

There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Fold 3 — eval_wer: 0.0131, eval_cer: 0.0021

===== Fold 4/4 =====


Step,Training Loss,Validation Loss,Wer,Cer
400,0.002900,0.005736,0.026756,0.004578
800,0.006000,0.004112,0.022074,0.003751
1200,0.001300,0.002291,0.008473,0.001685
1600,0.000100,0.001765,0.006912,0.001208


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Fold 4 — eval_wer: 0.0069, eval_cer: 0.0012


In [30]:
# (Optional) Final evaluation on your held-out test set:
final_trainer = Seq2SeqTrainer(
    model=model, args=training_args,
    train_dataset=train_data, eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
print("Test WER:", final_trainer.evaluate()["eval_wer"])
print("Test WER:", final_trainer.evaluate()["eval_cer"])
final_trainer.push_to_hub()

/tmp/ipykernel_624/2072296669.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  final_trainer = Seq2SeqTrainer(


Test WER: 0.2940391459074733
Test WER: 0.05782258834006552


events.out.tfevents.1746685350.060b9483f20e.624.0:   0%|          | 0.00/6.73k [00:00<?, ?B/s]

events.out.tfevents.1746693099.060b9483f20e.624.3:   0%|          | 0.00/453 [00:00<?, ?B/s]

events.out.tfevents.1746685440.060b9483f20e.624.2:   0%|          | 0.00/9.39k [00:00<?, ?B/s]

...

events.out.tfevents.1746708124.060b9483f20e.624.8:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

events.out.tfevents.1746715641.060b9483f20e.624.9:   0%|          | 0.00/453 [00:00<?, ?B/s]

events.out.tfevents.1746716133.060b9483f20e.624.10:   0%|          | 0.00/808 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kaarthu2003/whisper-small-telugu-k-fold-4/commit/43112f51808b048b9a94e931a75bc06237fabecf', commit_message='End of training', commit_description='', oid='43112f51808b048b9a94e931a75bc06237fabecf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kaarthu2003/whisper-small-telugu-k-fold-4', endpoint='https://huggingface.co', repo_type='model', repo_id='kaarthu2003/whisper-small-telugu-k-fold-4'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("kaarthu2003/whisper-small-telugu")
processor = WhisperProcessor.from_pretrained("kaarthu2003/whisper-small-telugu")

In [ ]:
print(test_data[0])

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Placeholder to store predictions and references
predictions = []
references = []

# Evaluate your model on the test data
for example in test_data:
    # Extract input features and labels
    input_features = example["input_features"]
    label = example["labels"]

    # Move input features to device
    input_features = torch.tensor(input_features).unsqueeze(0).to(device)

    # Generate predictions
    predicted_ids = model.generate(input_features)
    predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    # Check if label is an integer or needs formatting
    if isinstance(label, int):
        label = [label]  # Wrap the single integer in a list if necessary

    # Decode the label to get the reference text
    reference_text = processor.batch_decode([label], skip_special_tokens=True)[0]

    # Append to lists
    predictions.append(predicted_text)
    references.append(reference_text)

# Compute WER
wer = metric.compute(predictions=predictions, references=references)
print(f"Word Error Rate (WER): {wer:.2f}")
cer_metric = evaluate.load("cer")
cer = cer_metric.compute(predictions=predictions, references=references)
print(f"Character Error Rate (CER): {cer: .2f}")